# Transcriptomics Scratch Pad

In [91]:
using DataFrames, CSV, Statistics, CairoMakie

include("/Users/fomo/Documents/Kaizen/code/bioinformatics/modules/Plotify.jl")
include("/Users/fomo/Documents/Kaizen/code/bioinformatics/modules/GeoPreprocess.jl")

nothing

## Preprocessing

In [81]:
master = "../data/GEO"
geos = readdir(master)
meta = CSV.read(joinpath(master, geos[2], "metadata.csv"), DataFrame)

first(meta, 5)

Row,GSM_ID,Title,Source,Characteristics
,String15,String31,String31,String
1,GSM1220792,695_EV_0hr,hOB695_shEV_0hr,cell line: Normal bone cell line 695; phenotype: Parental; cell type: human osteoblast (hOB)
2,GSM1220793,695-EV_2hr,hOB695_shEV_2hr_IR,cell line: Normal bone cell line 695; phenotype: Parental; cell type: human osteoblast (hOB)
3,GSM1220794,695-EV_4hr,hOB695_shEV_4hr_IR,cell line: Normal bone cell line 695; phenotype: Parental; cell type: human osteoblast (hOB)
4,GSM1220795,695-EV_8hr,hOB695_shEV_8hr_IR,cell line: Normal bone cell line 695; phenotype: Parental; cell type: human osteoblast (hOB)
5,GSM1220796,695-EV_16hr,hOB695_shEV_16hr_IR,cell line: Normal bone cell line 695; phenotype: Parental; cell type: human osteoblast (hOB)


In [98]:
function parse_gse50532(raw::DataFrame)
    # 1) Derive control vs irradiated
    raw.group = ifelse.(occursin.("_IR", raw.Source),
                        "irradiated",
                        "control")

    # 2) Extract the number before “hr” via match
    raw.time_hr = [ begin
        m = match(r"_(\d+)hr", title)
        m === nothing ? missing : parse(Float64, m.captures[1])
      end
      for title in raw.Source
    ]

    # 3) Return only the columns you need
    return select(raw,
                  [:GSM_ID, :group, :time_hr, :cell_type])
end


parse_gse50532 (generic function with 1 method)

In [99]:
meta = parse_gse50532(meta)

ArgumentError: ArgumentError: column name :cell_type not found in the data frame

In [94]:
gse50532 = load_GSE(
    "GSE50532";
    basepath = master,
    parser = parse_gse50532
)

ErrorException: type RegexMatchIterator has no field captures